In [1]:
import os
import requests
import openai
from openai import OpenAI
from dotenv import load_dotenv
from bs4 import BeautifulSoup

### Connect to OpenAI

In [2]:
load_dotenv(override = True)
api_key= os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


### Create a Clinet

In [3]:
client = OpenAI()

In [4]:
message = "Hey! I am so excited to use a Frontier Model to help me"
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [{"role": "user", "content": message}]
)

print(response.choices[0].message.content)


That sounds great! Frontier models can be really useful for various applications, whether it's in economics, finance, machine learning, or any other field. How can I assist you with it today? Are you looking to understand how to use a specific frontier model, or do you have a particular application in mind?


In [5]:
!pip install PyPDF2

In [6]:
import requests
from io import BytesIO
from PyPDF2 import PdfReader

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Paper:
    def __init__(self, url):
        self.url = url
        self.title = url.split('/')[-1]  # Using filename as the title
        self.text = self._extract_text()

    def _extract_text(self):
        response = requests.get(self.url, headers=headers)  # 👈 Added headers here!
        if response.status_code != 200:
            raise Exception(f"Failed to download the paper. Status code: {response.status_code}")

        pdf_file = BytesIO(response.content)
        reader = PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:  # Only add if there is text extracted
                text += page_text + "\n"
        return text

paper = Paper("https://people.clas.ufl.edu/hager/files/BOSVS.pdf")
print(paper.title)
print(paper.text[:3000])  # Print first 1000 characters to avoid too much output




BOSVS.pdf
Comput Optim Appl
DOI 10.1007/s10589-012-9519-2
Bregman operator splitting with variable stepsize
for total variation image reconstruction
Yunmei Chen ·William W. Hager ·
Maryam Yashtini ·Xiaojing Ye ·Hongchao Zhang
Received: 25 January 2012
© Springer Science+Business Media New York 2012
Abstract This paper develops a Bregman operator splitting algorithm with variable
stepsize (BOSVS) for solving problems of the form min {φ(Bu) +1/2/bardblAu−f/bardbl2
2},
where φmay be nonsmooth. The original Bregman Operator Splitting (BOS) algo-
rithm employed a ﬁxed stepsize, while BOSVS uses a line search to achieve better
efﬁciency. These schemes are applicable to total variation (TV)-based image recon-
struction. The stepsize rule starts with a Barzilai-Borwein (BB) step, and increases
the nominal step until a termination condition is satisﬁed. The stepsize rule is related
This research was partly supported by National Science Foundation grants 1115568 and 1016204,
and by Ofﬁce of Nava

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a mathematics \
paper and provides a short concise summary, ignoring mathematics formulas. \
Respond in markdown."


In [8]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(paper):
    user_prompt = f"You are looking at a paper titled {paper.title}"
    user_prompt += "\nThe contents of this paper is as follows; \
    please provide a short summary of this paper in markdown.\
    Eliminate the mathematics formulas. Text is long, truncating \
    to 4000 characters for summarization. \n\n"
    user_prompt += paper.text
    return user_prompt

user_prompt_for(paper)

In [9]:
def summarize_paper(paper):
    paper = Paper(paper)
    response = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=[{"role": "user", "content": user_prompt_for(paper)}]
    )

    return response.choices[0].message.content.strip()


In [10]:
paper_link = "https://people.clas.ufl.edu/hager/files/BOSVS.pdf"

summary = summarize_paper(paper_link)
print("Summary:\n", summary)



Summary:
 # Summary of "Bregman Operator Splitting with Variable Stepsize for Total Variation Image Reconstruction"

**Authors**: Yunmei Chen, William W. Hager, Maryam Yashtini, Xiaojing Ye, Hongchao Zhang  
**Publication**: Comput Optim Appl, DOI: 10.1007/s10589-012-9519-2  
**Date**: January 25, 2012  

## Abstract
This paper presents a novel algorithm named Bregman Operator Splitting with Variable Stepsize (BOSVS) for minimizing optimization problems of the form involving nonsmooth functions. The traditional Bregman Operator Splitting (BOS) method utilizes a fixed stepsize, while BOSVS adopts a line search strategy to improve efficiency, particularly in applications related to total variation-based image reconstruction. The algorithm begins with a Barzilai-Borwein (BB) step and adapts the stepsize until a stopping condition is met. The performance of this approach is evaluated against other operator splitting methods, particularly in the context of partially parallel magnetic resona

# Excellent JOB! 